In [1]:
import os
import torch

import csv 

import pandas as pd
import numpy as np

In [ ]:
disease_names = np.array(['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 
                 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema',
                'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia', 'No Finding'])




In [3]:
data = []
names = []
target = []

data_sheet = pd.read_csv("/home/user/Chest_X_Ray_Data/CXR14/CXR8/Data_Entry_2017_v2020.csv")

print(data_sheet)

N = data_sheet.shape[0]

             Image Index          Finding Labels  Follow-up #  Patient ID  \
0       00000001_000.png            Cardiomegaly            0           1   
1       00000001_001.png  Cardiomegaly|Emphysema            1           1   
2       00000001_002.png   Cardiomegaly|Effusion            2           1   
3       00000002_000.png              No Finding            0           2   
4       00000003_001.png                  Hernia            0           3   
...                  ...                     ...          ...         ...   
112115  00030801_001.png          Mass|Pneumonia            1       30801   
112116  00030802_000.png              No Finding            0       30802   
112117  00030803_000.png              No Finding            0       30803   
112118  00030804_000.png              No Finding            0       30804   
112119  00030805_000.png              No Finding            0       30805   

        Patient Age Patient Gender View Position  OriginalImage[Width  \
0 

In [4]:
individual_lebels = []

all_labels = data_sheet['Finding Labels']

for i in range(N):
    label = all_labels[i] 
    if '|' in label:
        pass
    else:
        individual_lebels.append(label)

print(np.unique(individual_lebels))

['Atelectasis' 'Cardiomegaly' 'Consolidation' 'Edema' 'Effusion'
 'Emphysema' 'Fibrosis' 'Hernia' 'Infiltration' 'Mass' 'No Finding'
 'Nodule' 'Pleural_Thickening' 'Pneumonia' 'Pneumothorax']


In [5]:
g = np.where(np.array(['Atelectasis', 'Cardiomegaly', 'Effusion'])=='Cardiomegaly')[0]
print(g)
print(len(g))

[1]
1


In [6]:
filenames = {}
for disease in disease_names:
    filenames[disease] = []
    
print(filenames)

{'Atelectasis': [], 'Cardiomegaly': [], 'Effusion': [], 'Infiltration': [], 'Mass': [], 'Nodule': [], 'Pneumonia': [], 'Pneumothorax': [], 'Consolidation': [], 'Edema': [], 'Emphysema': [], 'Fibrosis': [], 'Pleural_Thickening': [], 'Hernia': [], 'No Finding': []}


In [7]:
all_files_0 = data_sheet['Image Index']
all_labels = data_sheet['Finding Labels']

full_labels_0 = np.zeros((N,len(disease_names)),dtype=np.float32)

for i in range(N):
    if i%10000 == 0:
        print('Completed ', i, ' of ', N)
    label = all_labels[i]
    for j in range(len(disease_names)):
        disease = disease_names[j]
        
        if disease in label:
            full_labels_0[i,j] = 1.0
                

Completed  0  of  112120
Completed  10000  of  112120
Completed  20000  of  112120
Completed  30000  of  112120
Completed  40000  of  112120
Completed  50000  of  112120
Completed  60000  of  112120
Completed  70000  of  112120
Completed  80000  of  112120
Completed  90000  of  112120
Completed  100000  of  112120
Completed  110000  of  112120


In [8]:
test_files_list = np.loadtxt("/home/user/Chest_X_Ray_Data/CXR14/CXR8/test_list.txt", dtype='str')

test_files = []
test_labels = []

all_files = []
full_labels = []

for file,label in zip(all_files_0,full_labels_0):
    if file in test_files_list:
        test_files.append(file)
        test_labels.append(label)
    else:
        all_files.append(file)
        full_labels.append(label)
        


In [9]:
test_labels = np.array(test_labels)
full_labels = np.array(full_labels)

print('Tarining: ', len(all_files), full_labels.shape)
print('test: ', len(test_files), test_labels.shape)

Tarining:  86524 (86524, 15)
test:  25596 (25596, 15)


In [12]:
full_idx_per_label = []
test_idx_per_label = []

total = 0

for i in range(full_labels.shape[1]):
    z = np.where(full_labels[:,i]>0.5)[0]
    full_idx_per_label.append(z)
    zz = np.where(test_labels[:,i]>0.5)[0]
    test_idx_per_label.append(zz)
    
    l = z.shape[0]+zz.shape[0]
    total+=l
    print(l, total)

11559 11559
2776 14335
13317 27652
19894 47546
5782 53328
6331 59659
1431 61090
5302 66392
4667 71059
2303 73362
2516 75878
1686 77564
3385 80949
227 81176
60361 141537


In [ ]:
from imageio import imread 
import PIL 
from PIL import Image 
from torchvision import transforms 

import torch
from torchvision import transforms

device = torch.device("cuda:1") # if torch.cuda.is_available() else "cpu")

import torch.nn as nn
import torch.nn.functional as F
model = torch.hub.load('pytorch/vision:v0.5.0', 'resnet50', pretrained=False)
model.fc = nn.Linear(2048, 15)
model.to(device)
print(model)

import torch.optim as optim

m = nn.Sigmoid()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:


preprocess = transforms.Compose([
    transforms.Resize(256),
    #transforms.CenterCrop(224),
    #transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


#all_files = data_sheet['Image Index']
#full_labels

loaded_images= []

for i in range(len(all_files)):

    file_to_load = '/home/user/Chest_X_Ray_Data/CXR14/CXR8/images/extracted/'+all_files[i]
    img = imread(file_to_load, as_gray=False, pilmode="RGB") 
    img = Image.fromarray(img)
    img = PIL.ImageOps.equalize(img)
    img = preprocess(img)
    loaded_images.append(img)


In [14]:
print(len(loaded_images))
print(loaded_images[0:2])

86524
[<PIL.Image.Image image mode=RGB size=256x256 at 0x7F2B0DD8A9B0>, <PIL.Image.Image image mode=RGB size=256x256 at 0x7F2B0DD8A7F0>]


In [15]:
augments = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def sample_images(images,full_labels, full_idx_per_label, device, n_sample=2):
    label_idx = []
    chosen_images = []
    
    for i in range(len(full_idx_per_label)):
        N = len(full_idx_per_label[i])
        chs = np.random.choice(full_idx_per_label[i], n_sample)
        label_idx.append(chs)
        augmented_images = []
        #print(chs)
        for idx in chs:
            img = augments(images[idx])
            augmented_images.append(img)

        augmented_images = torch.stack(augmented_images).to(device)
        chosen_images.append(augmented_images)
    
    chs = np.array(label_idx).reshape(-1)
    labels = torch.as_tensor(full_labels[chs]).to(device)
    augmented_images = torch.cat(chosen_images)
    
    return augmented_images, labels

G, label = sample_images(loaded_images,full_labels,full_idx_per_label,device=device,n_sample=2)
print(G.size())
print(label)

torch.Size([30, 3, 224, 224])
tensor([[1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.,

In [24]:

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)

In [32]:
for epoch in range(10):  # loop over the dataset multiple times
    
    scheduler.step()
    
    running_loss = 0.0
    for i in range(10000):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = sample_images(loaded_images, full_labels, full_idx_per_label,
                                       device=device, n_sample=2)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = m(model(inputs))
        #print(outputs.size(), labels.size())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 0.025
[1,  4000] loss: 0.024
[1,  6000] loss: 0.024
[1,  8000] loss: 0.024
[1, 10000] loss: 0.024
[2,  2000] loss: 0.024
[2,  4000] loss: 0.023
[2,  6000] loss: 0.023
[2,  8000] loss: 0.023
[2, 10000] loss: 0.023
[3,  2000] loss: 0.023
[3,  4000] loss: 0.022
[3,  6000] loss: 0.022
[3,  8000] loss: 0.022
[3, 10000] loss: 0.022
[4,  2000] loss: 0.022
[4,  4000] loss: 0.022
[4,  6000] loss: 0.022
[4,  8000] loss: 0.021
[4, 10000] loss: 0.022
[5,  2000] loss: 0.021
[5,  4000] loss: 0.021
[5,  6000] loss: 0.021
[5,  8000] loss: 0.021
[5, 10000] loss: 0.021
[6,  2000] loss: 0.021
[6,  4000] loss: 0.021
[6,  6000] loss: 0.021
[6,  8000] loss: 0.021
[6, 10000] loss: 0.020
[7,  2000] loss: 0.020
[7,  4000] loss: 0.020
[7,  6000] loss: 0.020
[7,  8000] loss: 0.020
[7, 10000] loss: 0.020
[8,  2000] loss: 0.020
[8,  4000] loss: 0.020
[8,  6000] loss: 0.020
[8,  8000] loss: 0.020
[8, 10000] loss: 0.020
[9,  2000] loss: 0.020
[9,  4000] loss: 0.020
[9,  6000] loss: 0.020
[9,  8000] 

In [33]:
torch.save(model.state_dict(), 'saved_model/cxr14_overlapping_labels_bceloss_resnet50_2.pth')

In [ ]:
model2 = torch.hub.load('pytorch/vision:v0.5.0', 'resnet50', pretrained=False)
model2.fc = nn.Linear(2048, 15)

model2.load_state_dict(torch.load('saved_model/cxr14_overlapping_labels_bceloss_resnet50_2.pth'))
model.to(device)
model2.eval()

In [35]:
preprocess_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


#test_files
#test_labels

test_images= []

for i in range(len(test_files)):

    file_to_load = '/home/user/Chest_X_Ray_Data/CXR14/CXR8/images/extracted/'+test_files[i]
    img = imread(file_to_load, as_gray=False, pilmode="RGB") 
    img = Image.fromarray(img)
    img = PIL.ImageOps.equalize(img)
    img = preprocess_test(img)
    test_images.append(img)



In [36]:
test_predict = []

for i in range(len(test_images)):
    img = torch.unsqueeze(test_images[i], dim=0).to(device)
    outputs = m(model(img)).cpu().detach().numpy()
    test_predict.append(outputs)

In [37]:
test_predict = np.squeeze(np.array(test_predict))

print(test_predict.shape, test_labels.shape)

(25596, 15) (25596, 15)


In [38]:
from sklearn.metrics import roc_auc_score, f1_score

acc = np.mean(test_labels == (test_predict>.5),axis=0)
print(acc.shape)

for i in range(len(disease_names)):
    print(disease_names[i], ',\n\t Accuracy : ', acc[i], 
          ' AUC: ', roc_auc_score(test_labels[:,i], test_predict[:,i]),
          'F1: ', f1_score(test_labels[:,i], 1.0*(test_predict[:,i]>0.5)))

(15,)
Atelectasis ,
	 Accuracy :  0.8717377715268011  AUC:  0.6160861892919653 F1:  0.0036418816388467377
Cardiomegaly ,
	 Accuracy :  0.9582356618221597  AUC:  0.783036319379689 F1:  0.0
Effusion ,
	 Accuracy :  0.818057508985779  AUC:  0.7083589957321912 F1:  0.00299721687004924
Infiltration ,
	 Accuracy :  0.7453117674636662  AUC:  0.6128366309220292 F1:  0.17699785380633756
Mass ,
	 Accuracy :  0.9317080793874043  AUC:  0.6424919081336642 F1:  0.0
Nodule ,
	 Accuracy :  0.9365916549460853  AUC:  0.5716111129230694 F1:  0.0
Pneumonia ,
	 Accuracy :  0.9783169245194562  AUC:  0.5594289509348811 F1:  0.0
Pneumothorax ,
	 Accuracy :  0.8958821690889202  AUC:  0.6602530407111702 F1:  0.0
Consolidation ,
	 Accuracy :  0.9290904828879513  AUC:  0.5835474601051398 F1:  0.0
Edema ,
	 Accuracy :  0.9638615408657603  AUC:  0.6838710511411253 F1:  0.0
Emphysema ,
	 Accuracy :  0.957298015314893  AUC:  0.611007506260133 F1:  0.0
Fibrosis ,
	 Accuracy :  0.98300515705579  AUC:  0.610113672546501